In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import os

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling2D, ZeroPadding2D, Reshape

Using TensorFlow backend.


In [2]:
def read_files(filetype):
    path = 'data/aclImdb/'
    file_list = []
    
    positive_path = path + filetype + '/pos/'
    for f in os.listdir(positive_path):
        file_list += [positive_path+f]
    
    negative_path = path + filetype + '/neg/'
    for f in os.listdir(negative_path):
        file_list += [negative_path+f]
    
    print('read', filetype, 'files:', len(file_list))
    
    all_labels = ([1] * 12500 + [0] * 12500)
    
    all_texts = []
    for fi in file_list:
        with open(fi, encoding='utf8') as file_input:
            all_texts += [rm_tags(' '.join(file_input.readlines()))]
    return all_labels, all_texts

In [3]:
import re 
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('', text)

In [4]:
y_train, train_text = read_files('train')
y_test, test_text = read_files('test')

read train files: 25000
read test files: 25000


In [5]:
#token = Tokenizer(num_words=2000)
token = Tokenizer(num_words=3800)
token.fit_on_texts(train_text)

In [6]:
X_train_seq = token.texts_to_sequences(train_text)
X_test_seq = token.texts_to_sequences(test_text)

X_train = sequence.pad_sequences(X_train_seq, maxlen=100) #Branch
X_test = sequence.pad_sequences(X_test_seq, maxlen=100)

In [7]:
X_train = sequence.pad_sequences(X_train_seq, maxlen=380)
X_test = sequence.pad_sequences(X_test_seq, maxlen=380)

In [8]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding

In [9]:
model = Sequential()

model.add(Embedding(output_dim=32, #Branch
                    input_dim=2000,
                    input_length=100))
model.add(Dropout(0.2))

In [10]:
model.add(Embedding(output_dim=32,
                    input_dim=3800,
                    input_length=380))
model.add(Dropout(0.2))

In [11]:
#model.add(Flatten())
#from keras.layers import SimpleRNN
#model.add(SimpleRNN(units=16))
from keras.layers import LSTM
model.add(LSTM(units=(300,3)))

TypeError: unsupported operand type(s) for +: 'int' and 'tuple'

In [ ]:
model.summary()

In [ ]:
model.add(Conv2D(filters=16, kernel_size=(2,2),
                activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
#######################
model.add(Dense(units=256,
                activation='relu'))
model.add(Dropout(0.35))

In [ ]:
model.add(Dense(units=1,
                activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
train_history = model.fit(X_train, y_train, batch_size=100,
                          epochs=10, verbose=2,
                         validation_split=0.2)

In [ ]:
from def_function import show_train_history
show_train_history(train_history, 'acc', 'val_acc')

In [ ]:
scores = model.evaluate(X_test,
                        y_test,
                        verbose=1)
scores[1]

In [ ]:
predict = model.predict_classes(X_test)

In [ ]:
predict[:10]

In [ ]:
predict_classes = predict.reshape(-1)
predict_classes[:10]

In [ ]:
def display_test_Sentiment(i):
    SentimentDict={1:'pos', 0:'neg'}
    print(test_text[i])
    print('label真實值:', SentimentDict[y_test[i]],
          '預測結果:', SentimentDict[predict_classes[i]])

In [ ]:
display_test_Sentiment(2)